# Simulating a Low-Pass Filter in a Circuit

Copyright 2024 Mike Augspurger, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)), based on an idea by Allen Downey

## Introduction

In experimentation, we often need to record a *dynamic signal*: that is, a signal that changes with time, like the vibration of a bridge, the velocity of wind, the pressure in a speaker.  These signals are often *complex*: they contain multiple dynamic signals all added together to create a complicated data set.   One way of simplifying such a data set is to *filter* the signal: if we know that the high frequency component of an input signal is some kind of unwanted "noise", we can take those high frequency components out using a *low pass filter*.

<br>

In this notebook, we will simulate the low-pass filter for input signals to determine how well they remove unwanted noise.  The input signals will be sinusoidal waves of this form:

<br>

$$ V_{in}(t) = A \cos (2 \pi f t) $$

<br>

where $A$ is the amplitude of the input signal (usually in units of volts), and $f$ is the frequency of the signal in Hz.   The frequency of a signal tells you how many times the signal oscillates in a second.  Here, for instance, is a time plot of a 10 Hz wave with an amplitude of 5 V:

<br><center>

<img src=https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_4/sinusoidal_wave.png width=500></center>

<br>

Notice that it oscillates 10 times in 1 second (count the peaks!) and has a amplitude that ranges from -5 to 5 V.

## Part 1: Setting up a system and state

The resistance and capacitance of the components in the circuit define the behavior of the circuit in response to changes. The amplitude and frequency of the input signal define the input values for the filter.   We want to put these values in a system dictionary.  But in doing so, we also want to calculate a couple parameters that are derive from these values:

* `omega` is the frequency of the input signal in radians/second.  We can use this value in numpy functions.  To calculate `omega`:

$$\omega = 2\pi f$$

<br>

* `tau` is the time constant for this circuit, which is roughly the time it takes to charge (i.e. fill) the capacitor.  As we saw with the differential equation, the coefficient we want is the inverse of the time constant:

$$\tau_inv = \frac{1}{RC}$$

<br>

* Finally, we want to end the simulation after 8 cycles of the input signal.  So define `t_end` as the time required to do that (think about how long it takes to complete one cycle for a signal with a given frequency; note that the units $Hz$ is equivalent to $\frac{1}{s}$.

Adapt the `make_system` and `make_state` from the coffee notebooks below to our new purpose:

In [ ]:
import pandas as pd
import numpy as np

# Define parameters
R1 = 1.0e6   # ohm
C1 = 1.0e-9  # farad
A = 5.0      # volt
f = 1000.0   # Hz

def make_system(R1,C1,A,f):
    # Calculate tau_inv, omega, and t_end in here
    # dt should be much smaller than t_end (try 10,000 times smaller): you can calculate that here too
    return dict(T_init=T_init, T_final=0, volume=volume,
                  r=r, t_end=t_end, T_env=T_env, T_goal=T_goal,
                  t_0=0,  dt=1)

Now make a state by adapting the coffee code again.  What is the state variable?  It makes sense to give the state variable an initial value of 0:

In [ ]:
def make_state(system):
    return pd.Series(dict(temp=system['T_init']), name='State object')

Finally, call the two functions and make sure their output makes sense:

In [ ]:
system1 = make_system(R1,C1,A,f)
state1 = make_state(system1)
print(system1)
print(state1)

## Part 2: The change function

You'll need a change function that incorporates the differential equation from Notebook 2.4.7.  You'll have to calculate $V_{in}$ using the equation at the top of this notebook.  `Omega` should come in handy!

<br>

As usual, the function should return the change in the state variable for that time step.




In [ ]:
# Here's the change_func from the coffee code
def change_func(t, state, system):
    r, T_env, dt = system['r'], system['T_env'], system['dt']
    deltaT =  -r * (state.temp - T_env) * dt
    return deltaT

In [ ]:
# Now test your change function: the return value should
# change depending on the value of t that you enter!
dT_coffee = change_func(0,state,system)
print(dT_coffee)

## Part 3:  Run_simulation


Make adjustments to run simulation as necessary.  The tricky change here is that you'll want to print both $V_{in}$ and $V_{out}$.  You'll find $V_{out}$ in the `for` loop, but you can calculate $V_{in}$ directly using `t_array`.  Look back at the "Historical Population" exercise from 2 weeks back to remember how to do that!

In [ ]:
def run_simulation(system, change_func):
    t_array = np.arange(system['t_0'], system['t_end'], system['dt'])
    n = len(t_array)
    state = make_state(system)
    results = pd.Series(index=t_array,dtype=object)
    results.iloc[0] = state.temp

    for i in range(n-1):
        t = t_array[i]
        delta = change_func(t, state, system)
        state.temp = state.temp + delta
        results.iloc[i+1] = state.temp

    system['T_final'] = results.iloc[-1]
    return results

In [ ]:
# Plot your results and make sure they make sense

results.plot(xlabel='time (s)', ylabel='Temperature (degrees C',
                 title='Temperature of Cooling Object', label='Aluminum', legend=True)


If things have gone according to plan, the amplitude of the output signal should be about 0.8 V.  Try out different values for R, C, and f to see how the different filters behave.

## Part 4: Defining a metric

One way to think about the behavior of a low pass filter is to consider the *amplitude ratio* for a given frequency: what is the ratio of the amplitude of the input signal to that of the output signal.  An amplitude ratio of 1.0 means that the signal was not filtered at all, while a ratio of 0.0 implies that the signal was completely removed.

<br>

See if you can change `run_simulation()` above so that it produces and returns the amplitude ratio for this system.   You may need use a numpy, pandas, or math function to help you out (use the internet to help).  The amplitude ratio for this system should be about 0.15.

<br>



## Part 5: Sweep through frequencies to plot amplitude ratio

Now that we have a metric, we want to show how our filter responds to different independent variables.   It is common when discussing a filter to plot the amplitude ratios as a function of the frequency: to "sweep" through frequencies and find how much of the signal is filtered.

<br>

To do that, you'll need to do a couple things:

* Create a frequency array that runs from 10 Hz to 100,000 Hz. Since the behavior of the system changes as the frequency doubles and re-doubles, we want an array that grows exponentially: this has been done in the cell below for you.

<br>

* Create a empty Series called `sweep` that will hold the metric for each frequency.   This will look a lot like the empty `results` folder you created in `run_simulation()`

<br>

* Set up a `for` loop that loops through the frequencies, and for each frequency runs a simulation and records the amplitude ratio in the empty Series that you created.

In [2]:
import numpy as np

# Here's a array of frequencies that grows exponentially rather than linearly
f_array = np.array([10.0,25.0,50.0,100.0,250.0,500.0,1000.0,2500.0,5000.0,10000.0,100000.0])



array([4, 5, 6])

In [ ]:
# Now plot your results
# Notice the keyword argument "logx = True": this makes the x-axis increase
# logarithmically which makes the data easier to read because of its exponential behavior
sweep.plot(title = "Amplitude ratio for Filter with tau = 0.001 s",
           xlabel = "Frequency (Hz)", ylabel = "Amplitude ratio", logx = True);

✅ ✅  Analysis Question:  The "cutoff" frequency is the frequency above which the input signal is significantly reduced.  What would you say the cut off frequency of this filter is?  What makes this hard to define?

✅ ✅ Put your answer here